# This notebook parses data from website, exactly from pricelist, then it compares and merges with data taken from the directory by adding comments where needed and sets style to the file and exports as an excel file.

Firstly, import some libraries needed to work in this notebook.

In [1]:
import pandas as pd 
import numpy as np
import openpyxl
import xlsxwriter 
from bs4 import BeautifulSoup as bs
import requests
from openpyxl import Workbook

Saving cities as a dictionary

In [2]:
cities = {
    'Астана' : 'astana', 
    'Актау' : 'aktau',
    'Актобе' : 'aktobe',
    'Алматы' : 'almaty',
    'Атырау' : 'atyrau',
    'Караганда' : 'karaganda',
    'Кокшетау' : 'kokshetau',
    'Костанай' : 'kostanay',
    'Кызылорда' : 'kyzylorda',
    'Павлодар' : 'pavlodar',
    'Петропавловск' : 'petropavlovsk',
    'Талдыкорган' : 'taldykorgan',
    'Тараз' : 'taraz',
    'Туркестан' : 'turkestan',
    'Уральск' : 'uralsk',
    'Усть-Каменогорск' : 'ust-kamenogorsk',
    'Шымкент' : 'shymkent'
}

Defining the URL as an address for pricelist

In [3]:
URL = 'url'

Setting variable city_var as a name of the city in cyrillic

In [4]:
city_var = 'city'

Setting paths and names of files for input and output 

In [5]:
path = '..\\data\\by_cities\\astana\\'
input_file = path + 'input_file'
output_file = path + 'output_file'
ext = '.xlsx'

In [6]:
workbook = Workbook()
workbook.save(output_file)
writer = pd.ExcelWriter(output_file, engine = 'xlsxwriter')

Reading data from the directory with pricelist data in LIS

In [7]:
df_pl_lis = pd.read_excel(path + 'file_to_merge', sheet_name=city_var)
df_pl_lis.rename(columns={'Код': 'Код_услуги', 'Название': 'Название_ЛИС', 'Группа услуг': 'Подразделение'}, inplace=True)

df_pl_lis.shape

(3506, 3)

Reading data for recent months from the LIS 

In [8]:
df_lis = pd.read_excel(input_file)

df_lis.shape

(999, 6)

Function for parsing data from the city's pricelist

In [9]:
def parse_city(city_var):
    
    content = requests.get(URL + cities[city_var])
    
    soup = bs(content.text, 'lxml')
    
    service_rows = soup.find_all('div', {'class' : 'service-row'})
    print('Количество всех услуг по городу ', city_var, ': ', len(service_rows))
    
    records = []
    
    for r in service_rows:
    
        service_name = r.find('div', {'class' : 'service-row__item service-title wide-title-cell'}).text #название услуги
        
        service_code = r.find('span', {'class' : 'add-to-cart'})['data-code'] #код услуги
        service_code = service_code.replace('В', 'B').replace(' ', '').replace('С', 'C').replace('Н', 'H').replace('М', 'M').replace('Т', 'T').replace('К', 'K').replace('А', 'A').replace('О', 'O').replace('Р', 'P').replace('р', 'p').replace('а', 'a').replace('', '')
        
        time = r.find('div', {'class': 'service-row__item period normal'}).text #указанный срок
        
        if time:
            
            time = time.replace(' ', '').replace('д.', '').replace('до', '').replace('от', '').replace('дней', '')
        
            if len(time.split('-')) == 2:
                time1, time2 = time.split('-')
            
            else:
                time1 = time2 = time
        
        else:
            time1 = time2 = time
        
        
        records.append({
            'Код_услуги' : service_code,
            'Услуга' : service_name,
            'ПЛ_мин' : time1,
            'ПЛ_макс' : time2
        })
        
        #print(len(records))
    
    df_parsed = pd.DataFrame(records)
    
    return df_parsed

Function for finding if service code is not in LIS pricelist directory

In [10]:
def check_codes(df1, df2):
    df1_codes = df1['Код_услуги'].tolist()
    df2_codes = df2['Код_услуги'].tolist()

    not_in_df2 = []

    for i in df1_codes:
        if i not in df2_codes:
            not_in_df2.append(i)
            
    return not_in_df2

Function for coloring rows with difference

In [11]:
def color_rows(row):
    
    min_1 = row['ПЛ_мин']
    min_2 = row['ЛИС_мин']
    qty = row['Количество']
    
    if qty >= 100:
        if min_1 and min_2 and pd.notnull(min_1) and pd.notnull(min_2):
    
            if float(min_1) > float(min_2):
                color = '#83ff83' #green
        
            elif float(min_1) < float(min_2):
                color = '#ff5050' #red
        
            else:
                color = 'white'
            
        else:
            color = 'white'
    
    else:
        color = 'white'
            
    return ['background-color: {}'.format(color) for r in row] #; border-style: solid; border: 1px solid #808080; grey;

The below code is like main, where the all operations and functions applied

In [12]:
df_parsed = parse_city(city_var)
    
print('Количество строк и столбцов записанных в таблицу: ', df_parsed.shape)

pl_lis_codes = df_pl_lis['Код_услуги'].tolist()
lis_codes = df_lis['Код_услуги'].tolist()

df_merged = pd.merge(df_parsed, df_pl_lis, how = 'left')

df_merged['Комментарий'] = ''
df_merged['Примечание'] = ''

for i, r in df_merged.iterrows():
    
    if r['Код_услуги'] not in pl_lis_codes:
        df_merged.at[i, 'Комментарий'] = 'нет в ЛИСе'
    else:
        if r['Услуга'] != r['Название_ЛИС']:
            df_merged.at[i, 'Примечание'] = 'не совпадает наименование'
        
    if r['Код_услуги'] not in lis_codes:
        df_merged.at[i, 'Комментарий'] = 'не заказывали'

final = pd.merge(df_merged, df_lis, how = 'left')

final = final[['Подразделение', 'Код_услуги', 'Услуга', 'Название_ЛИС', 'ПЛ_мин', 'ПЛ_макс', 'ЛИС_мин', 'ЛИС_макс', 'Количество', 'Комментарий', 'Примечание']]
print('Размер итогового датафрейма: ', final.shape)

final['Количество'] = final['Количество'].fillna(0)
final['ЛИС_мин'] = final['ЛИС_мин'].replace(0, 1).replace('0', '1')
final['ЛИС_макс'] = final['ЛИС_макс'].replace(0, 1).replace('0', '1')

styled = final.style.apply(color_rows, axis=1)

styled = styled.set_properties(**{'border-color': 'gray', 'border-width': '1px', 'border-style': 'solid'})
    
styled.to_excel(writer, index=False)
    
print('Данные по городу ', city_var, ' были выгружены.')
print()

Количество всех услуг по городу  Астана :  1380
Количество строк и столбцов записанных в таблицу:  (1380, 4)
Размер итогового датафрейма:  (1402, 11)
Данные по городу  Астана  были выгружены.



In [13]:
writer.close()
print('The task is completed!')

The task is completed!
